In [1]:
import math

import numpy as np
from scipy.sparse import dia_matrix
from scipy.linalg import expm

import matplotlib.pyplot as plt

In [4]:
def make_init(a_matrix, samples, init_temp):
    '''returns an initial state vector'''

    vec = np.zeros((a_matrix.shape[0], 1), dtype=float)

    assert samples == 5 or (samples >= 10 and samples % 10 == 0), "init region isn't evenly divided by discretization"

    # maximum z point for initial region is 0.2 for 5 samples and 0.1 otherwise
    max_z = samples // 10 + 1 if samples >= 10 else 2 * samples // 10 + 1

    for z in range(max_z):
        zoffset = z * samples * samples

        for y in range(2 * samples // 10 + 1):
            yoffset = y * samples

            for x in range(4 * samples // 10 + 1):
                index = x + yoffset + zoffset

                vec[index] = init_temp

    return vec

In [5]:
def heat3d_dia(samples, diffusity_const, heat_exchange_const):
    'fast dia_matrix construction for heat3d dynamics'

    samples_sq = samples**2
    dims = samples**3
    step = 1.0 / (samples + 1)

    a = diffusity_const * 1.0 / step**2
    d = -2.0 * (a + a + a)

    data = np.zeros((7, dims))
    offsets = np.array([-samples_sq, -samples, -1, 0, 1, samples, samples_sq], dtype=float)

    # element with z = -1
    data[0, :-samples_sq] = a

    # element with y = -1
    for s in range(samples):
        start = s * samples_sq
        end = (s + 1) * (samples_sq) - samples
        data[1, start:end] = a

    # element with x = -1
    for s in range(samples_sq):
        start = s * samples
        end = (s + 1) * (samples) - 1
        data[2, start:end] = a

    #### diagonal element ####
    data[3, :] = d     # (prefill)

    # adjust when z = 0 or z = samples-1
    data[3, :samples_sq] += a
    data[3, -samples_sq:] += a

    # adjust when y = 0 or y = samples-1
    for z in range(samples):
        z_offset = z * samples_sq

        data[3, z_offset:z_offset + samples] += a
        data[3, z_offset + samples_sq - samples:z_offset + samples_sq] += a

    # adjust when x = 0 (and add diffusion term when x = samples-1)
    for z in range(samples):
        for y in range(samples):
            offset = z * samples_sq + y * samples

            data[3, offset] += a

            data[3, offset + samples - 1] += a / (1+heat_exchange_const * step)

    #### end diagnal element ####
    # element with x = +1
    for s in range(samples_sq):
        start = 1 + s * samples
        end = (s + 1) * samples
        data[4, start:end] = a

    # element with y = +1
    for s in range(samples):
        start = s * samples_sq + samples
        end = (s + 1) * (samples_sq)
        data[5, start:end] = a

    # element with z = +1
    data[6, samples_sq:] = a

    rv = dia_matrix((data, offsets), shape=(dims, dims))
    assert np.may_share_memory(rv.data, data)     # make sure we didn't copy memory

    return rv

In [36]:
# parameters
samples_per_side = 5
diffusity_const = 0.01
heat_exchange_const = 0.5

a_matrix = heat3d_dia(samples_per_side, diffusity_const, heat_exchange_const)

In [37]:
import scipy.io

In [38]:
a_matrix

<125x125 sparse matrix of type '<class 'numpy.float64'>'
	with 813 stored elements (7 diagonals) in DIAgonal format>

In [39]:
scipy.io.savemat('a_matrix.mat', {'a_matrix': a_matrix})